## Setup

In [1]:
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(model = "llama3.1:8b-instruct-fp16")

In [2]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model = "llama3.1:8b-instruct-fp16")

#### Milvus vs. In-memory Vector Store

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [3]:
from langchain_milvus import Milvus

vector_store = Milvus(embedding_function=embeddings, auto_id = True) # Server on port 19530

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_PROJECT"] = "rag"

 ········


#### Dependencies

In [12]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph langchain-community beautifulsoup4

&emsp;

## Chains

In [13]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [14]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

ADD GITIGNORE & FIX FILTER - NOT CORRECT CODE FOR MILVUS!!!